
## Transformation for each table

In [0]:
# tables name

display(dbutils.fs.ls('/mnt/bronze/SalesLT/'))

path,name,size,modificationTime
dbfs:/mnt/bronze/SalesLT/Address/,Address/,0,1698590335000
dbfs:/mnt/bronze/SalesLT/Customer/,Customer/,0,1698590332000
dbfs:/mnt/bronze/SalesLT/CustomerAddress/,CustomerAddress/,0,1698590332000
dbfs:/mnt/bronze/SalesLT/Product/,Product/,0,1698590329000
dbfs:/mnt/bronze/SalesLT/ProductCategory/,ProductCategory/,0,1698590333000
dbfs:/mnt/bronze/SalesLT/ProductDescription/,ProductDescription/,0,1698590332000
dbfs:/mnt/bronze/SalesLT/ProductModel/,ProductModel/,0,1698590335000
dbfs:/mnt/bronze/SalesLT/ProductModelProductDescription/,ProductModelProductDescription/,0,1698590332000
dbfs:/mnt/bronze/SalesLT/SalesOrderDetail/,SalesOrderDetail/,0,1698590331000
dbfs:/mnt/bronze/SalesLT/SalesOrderHeader/,SalesOrderHeader/,0,1698590329000


In [0]:
tables_name = []
dbfs_path = dbutils.fs.ls('/mnt/bronze/SalesLT/')
for col in dbfs_path:
    tables_name.append(col.name.split('/')[0])
tables_name

['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader']

In [0]:
for i in tables_name:
    path = '/mnt/bronze/SalesLT/'+i +'/' 
    df = spark.read.format('parquet').load(path)
    column = df.columns

for col in column:
    if 'Date' in col or 'date ' in col:
        print(col)

OrderDate
DueDate
ShipDate
ModifiedDate


In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

# # tranform above date columns 
# for i in tables_name:
#     path = '/mnt/bronze/SalesLT/'+i +'/' 
#     df = spark.read.format('parquet').load(path)
#     column = df.columns
#     for col in column:
#         if 'Date' in col or 'date ' in col:
            # df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()),'UTC'), 'yyyy-MM-dd'))

In [0]:
# Complete code and save all tables transformation into silver cointaner 
for i in tables_name:
    path = '/mnt/bronze/SalesLT/'+i +'/' 
    df = spark.read.format('parquet').load(path)
    column = df.columns
    for col in column:
        if 'Date' in col or 'date ' in col:
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()),'UTC'), 'yyyy-MM-dd'))
    output_path = '/mnt/silver/SalesLT/'+i+'/'
    df.write.format('delta').mode('overwrite').save(output_path)